# Automated ML


In [1]:
!pip show azureml-core

Name: azureml-core
Version: 1.24.0.post2
Summary: Azure Machine Learning core packages, modules, and classes
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages
Requires: ndg-httpsclient, azure-common, pyopenssl, azure-mgmt-storage, PyJWT, msrest, requests, azure-mgmt-authorization, contextlib2, msrestazure, docker, python-dateutil, azure-graphrbac, backports.tempfile, azure-mgmt-resource, cryptography, jsonpickle, azure-mgmt-keyvault, SecretStorage, urllib3, jmespath, pytz, adal, pathspec, ruamel.yaml, azure-mgmt-containerregistry
Required-by: azureml-widgets, azureml-train-core, azureml-train-automl-runtime, azureml-train-automl-client, azureml-tensorboard, azureml-telemetry, azureml-sdk, azureml-pipeline-core, azureml-opendatasets, azureml-mlflow, azureml-interpret, azureml-defaults, azureml-datadrift

In [2]:
!pip install --upgrade azureml-sdk[automl]

Requirement already up-to-date: azureml-sdk[automl] in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.24.0)


In [3]:
!pip install --upgrade azureml-core

Requirement already up-to-date: azureml-core in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.24.0.post2)


In [4]:
import json
import sys
import os
import numpy as np
import pandas as pd
import shutil
import joblib
import requests

from sklearn.model_selection import train_test_split

from TrainCovid19Infections import clean_data

from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.run import Run
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails

from azureml.train.automl import AutoMLConfig

from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.train.automl import constants

## Initialize Workspace

In [5]:
# Get current workspace from config
ws = Workspace.from_config()
    
ws.get_details()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

project_folder = './capstone-project'
# choose a name for experiment
experiment_name = 'Covid19VaccinationExperiment'
experiment=Experiment(ws, experiment_name)
experiment

Workspace name: OptimizePipeline
Azure region: eastus2
Subscription id: c04b3d3f-4994-454d-96ff-aa3f2050b57f
Resource group: testingMLFunctionnalities


Name,Workspace,Report Page,Docs Page
Covid19VaccinationExperiment,OptimizePipeline,Link to Azure Machine Learning studio,Link to Documentation


## Create Cluster

Get cluster if it exists else create one

In [6]:
# Create compute cluster
cpu_cluster_name = "Covid19Cluster"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('A cluster with the same name already exists. If you are trying to create a new one please use a new cluster name')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4,identity_type="SystemAssigned")
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)
# Get a detailed status for the current cluster. 
print(cpu_cluster.get_status().serialize())

A cluster with the same name already exists. If you are trying to create a new one please use a new cluster name
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-20T22:06:00.792000+00:00', 'errors': None, 'creationTime': '2021-03-20T15:13:34.812815+00:00', 'modifiedTime': '2021-03-20T15:13:52.240694+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

### Overview

I Chose a COVID-19 World Vaccination Dataset that holds a track of the world vaccination including the name of the country, Which vaccines have been used by country, and how many have been vaccinated by Country.

Since the covid-19 vaccination is among the hottest subjects in the world, and as a member of the society being interested in such statistic calculations can help further scientists or even regular people to better understand the global effect of this vaccine all over the world.

I used Kaggle's API to download the Dataset.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [7]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Covid19InfectionsDataset"
description_text = "Covid19 Vaccination DataSet from Github"
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./github/owid-covid-data.csv'],
                       target_path ='train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)
if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    original_path = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
    ds = TabularDatasetFactory.from_delimited_files(original_path, infer_column_types=True, separator=',', header=True)
    
    
    #ds = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/country_vaccinations.csv')])
    
    #Register Dataset in Workspace
    dataset = ds.register(workspace=ws,
                          name=key,
                          description=description_text)

df = dataset.to_pandas_dataframe().fillna(0)
df.describe()

Uploading an estimated of 1 files
Uploading ./github/owid-covid-data.csv
Uploaded ./github/owid-covid-data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
count,7.600800e+04,76008.000000,76008.000000,7.600800e+04,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000,...,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000,76008.000000
mean,6.610629e+05,5121.867198,5056.453239,1.700790e+04,114.355726,113.112097,7828.250439,66.352370,65.222400,160.007469,...,27.703192,7.881587,5.024407,17414.585578,236.143373,7.253355,23.491059,2.548535,69.503854,0.667584
std,4.676108e+06,32165.477705,31613.208461,1.076847e+05,674.708492,652.071812,15579.320551,168.173557,140.944476,314.049383,...,12.417608,6.483054,4.370750,19610.341599,133.720476,4.293250,33.330064,2.519366,17.536964,0.246658
min,0.000000e+00,-74347.000000,-6223.000000,0.000000e+00,-1918.000000,-232.143000,0.000000,-2153.437000,-276.825000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.710000e+02,1.000000,4.429000,1.000000e+01,0.000000,0.000000,122.698250,0.061000,0.739000,1.589750,...,19.600000,3.008000,1.783000,2896.913000,140.448000,4.610000,0.000000,0.700000,66.470000,0.555000
50%,7.783000e+03,55.000000,62.714000,1.290000e+02,1.000000,0.857000,1032.970000,5.861000,7.511000,19.535000,...,29.000000,5.440000,3.212000,10727.146000,233.070000,6.930000,0.000000,2.000000,74.160000,0.737000
75%,8.658650e+04,660.000000,665.286000,1.700250e+03,11.000000,11.429000,7132.510500,55.749000,61.828250,138.043250,...,38.000000,13.260000,8.353000,25063.846000,318.949000,9.750000,47.782000,3.600000,78.490000,0.828000
max,1.223157e+08,880902.000000,739564.429000,2.701445e+06,17895.000000,14424.000000,147945.383000,8652.658000,2648.773000,2327.774000,...,48.200000,27.049000,18.493000,116935.600000,724.417000,30.530000,98.999000,13.800000,86.750000,0.957000


In [8]:
# preview the first 10 rows of the dataset
df.head(10)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
5,AFG,Asia,Afghanistan,2020-02-29,1.0,0.0,0.143,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
6,AFG,Asia,Afghanistan,2020-03-01,1.0,0.0,0.143,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
7,AFG,Asia,Afghanistan,2020-03-02,1.0,0.0,0.000,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
8,AFG,Asia,Afghanistan,2020-03-03,2.0,1.0,0.143,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511
9,AFG,Asia,Afghanistan,2020-03-04,4.0,2.0,0.429,0.0,0.0,0.0,...,1803.987,0,597.029,9.59,0,0,37.746,0.5,64.83,0.511


In [9]:
# Use the clean_data function to clean your data.
x, y = clean_data(df)
data = pd.concat([x,y],axis=1)
data.head()

,date,total_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,iso_code_ZMB,iso_code_ZWE,continent_0,continent_Africa,continent_Asia,continent_Europe,continent_North America,continent_Oceania,continent_South America,new_cases
0,737479,1.0,0.0,0.0,0.0,0.0,0.026,0.026,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1,737480,1.0,0.0,0.0,0.0,0.0,0.026,0.000,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,737481,1.0,0.0,0.0,0.0,0.0,0.026,0.000,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
3,737482,1.0,0.0,0.0,0.0,0.0,0.026,0.000,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
4,737483,1.0,0.0,0.0,0.0,0.0,0.026,0.000,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0


In [10]:
# Split data into train and test sets.
training_data,validation_data = train_test_split(data,test_size = 0.3,random_state = 42,shuffle=True)

In [11]:
# Create necessary folders
if "automl_training" not in os.listdir():
    os.mkdir("./automl_training")
if "data" not in os.listdir():
    os.mkdir("./data")
if "outputs" not in os.listdir():
    os.mkdir("./outputs")
if "training" not in os.listdir():
    os.mkdir("./training")
# store training_dataset into it using datastore
script_folder = './automl_training/'    
os.makedirs(script_folder, exist_ok=True)
shutil.copy('TrainCovid19Infections.py', script_folder)
project_folder = './pipeline-project'

## AutoML Configuration
TODO: Explain why you chose the automl settings and cofiguration you used below.
The settings used below refers to a classification task within a number of settings chosen based on the existing workspace and cluster configuration restrictions 

In [12]:
#convert the training dataset to a CSV file and store it under the training folder
training_data.to_csv('training/training_data.csv')
#Create an experiment for the AutoML testing script
exp = Experiment(workspace=ws, name="Covid19AutoMlExperiment")

# Get the dataset from the data folder
datastore.upload_files(files = ['training/training_data.csv'],
                       target_path ='./data/',
                       overwrite = True,
                       show_progress = True)
training_dataset = TabularDatasetFactory.from_delimited_files(path=[(datastore,('./data/training_data.csv'))])
#training_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/country_vaccinations.csv')])
automl_settings = {
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
    "enable_early_stopping": True,
    "experiment_timeout_hours": 1.0,
    "max_concurrent_iterations": 3,
}
automl_config = AutoMLConfig(task = 'classification',
                             compute_target = cpu_cluster,
                             training_data = training_dataset,
                             label_column_name = 'new_cases',
                             featurization= 'auto',
                             path=project_folder,
                              model_explainability=True,
                             debug_log = "Covid_automl_errors.log",
                             **automl_settings)

Uploading an estimated of 1 files
Uploading training/training_data.csv
Uploaded training/training_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [13]:
# Experiment Submission
tag = {"Covid19Infections": "Capstone project: Covid19 AutoML Experiment"}
remote_run = experiment.submit(automl_config,tags=tag, show_output=True)

Running on remote.
No run_configuration provided, running on Covid19Cluster with default configuration
Running on remote compute: Covid19Cluster
Parent Run ID: AutoML_44604385-5587-47cb-8037-b95f55ac0462


ERROR: Setup iteration failed: {"run_error": {"exception": {"message": "DataException:\n\tMessage: The ExperimentTimeout should be set more than {minimum} minutes with an input data of rows*cols({rows}*{columns}={total}), and up to {maximum}.\n\tInnerException: None\n\tErrorResponse \n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"The ExperimentTimeout should be set more than {minimum} minutes with an input data of rows*cols({rows}*{columns}={total}), and up to {maximum}.\",\n        \"target\": \"experiment_timeout_minutes\",\n        \"inner_error\": {\n            \"code\": \"BadArgument\",\n            \"inner_error\": {\n                \"code\": \"ArgumentOutOfRange\"\n            }\n        },\n        \"reference_code\": \"315a3631-c201-44a1-931

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?
Supervised Learning in general is based on labled 

In [14]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



ERROR: Setup iteration failed: {"run_error": {"exception": {"message": "DataException:\n\tMessage: The ExperimentTimeout should be set more than {minimum} minutes with an input data of rows*cols({rows}*{columns}={total}), and up to {maximum}.\n\tInnerException: None\n\tErrorResponse \n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"The ExperimentTimeout should be set more than {minimum} minutes with an input data of rows*cols({rows}*{columns}={total}), and up to {maximum}.\",\n        \"target\": \"experiment_timeout_minutes\",\n        \"inner_error\": {\n            \"code\": \"BadArgument\",\n            \"inner_error\": {\n                \"code\": \"ArgumentOutOfRange\"\n            }\n        },\n        \"reference_code\": \"315a3631-c201-44a1-931f-dad7a14b7ec4\"\n    }\n}", "exception_class": "DataException"}, "traceback": "  File \"setup_run.py\", line 72, in execute\n    verifier=verifier\n  File \"experiment_preparation_phase.py\", line 47, in

{'runId': 'AutoML_44604385-5587-47cb-8037-b95f55ac0462',
 'target': 'Covid19Cluster',
 'status': 'Failed',
 'startTimeUtc': '2021-03-20T23:03:09.924846Z',
 'endTimeUtc': '2021-03-20T23:16:48.505654Z',
 'error': {'error': {'code': 'UserError',
   'message': 'The ExperimentTimeout should be set more than 60 minutes with an input data of rows*cols(53205*261=13886505), and up to 10,000,000.',
   'messageFormat': 'The ExperimentTimeout should be set more than {minimum} minutes with an input data of rows*cols({rows}*{columns}={total}), and up to {maximum}.',
   'messageParameters': {'minimum': '60',
    'maximum': '10,000,000',
    'rows': '53205',
    'columns': '261',
    'total': '13886505'},
   'referenceCode': '315a3631-c201-44a1-931f-dad7a14b7ec4',
   'detailsUri': 'https://aka.ms/azureml-known-errors',
   'target': 'experiment_timeout_minutes',
   'details': [],
   'innerError': {'code': 'BadArgument',
    'innerError': {'code': 'ArgumentOutOfRange'}}},
  'time': '0001-01-01T00:00:00.

In [15]:
# List best models of HyperDrive Run and AutoML Run to compare the accuracy of the models and choose the best among them to deploy
for model in Model.list(ws):
    print(model.name)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print('\t',tag_name,':',tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print('\t',prop_name,':',prop)
    print("\n")

automl_covid19_model


HyperDrCovid19Model


HyperDrCovid19Model


HyperDrCovid19Model


HyperDrCovid19Model


HyperDrCovid19Model


HyperDrCovid19Model


HyperDrCovid19Model


HyperDrCovid19Model


HyperDrCovid19Model


HyperDrCovid19Model


HyperDrCovid19Model


HyperDrCovid19Model


HyperDrCovid19Model


HyperDrCovid19Model


HyperDrCovid19Model


sklearnBankmarketingModel


sklearnBankmarketingModel


automl_covid19_model




## Best Model

explaining_model_run_id = remote_run.id
print(explaining_model_run_id)
explaining_model_run = Run(experiment=experiment, run_id=explaining_model_run_id)
explaining_model_run.wait_for_completion()

In [17]:
# Retrieve and save best model.
best_automl_run, best_automl_model = remote_run.get_output()

AutoMLException: AutoMLException:
	Message: Could not find a model with valid score for metric 'accuracy'. Please ensure that at least one run was successfully completed with a valid score for the given metric.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Could not find a model with valid score for metric 'accuracy'. Please ensure that at least one run was successfully completed with a valid score for the given metric.",
        "target": "metric",
        "inner_error": {
            "code": "NotFound",
            "inner_error": {
                "code": "ModelMissing"
            }
        }
    }
}

In [ ]:
# Get the metrics of the best selected run
best_run_metrics = best_automl_run.get_metrics()

# Print all metrics of the best run model
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)
    
print(best_automl_model._final_estimator)

In [ ]:
joblib.dump(best_automl_model, filename="outputs/automl-model.pkl")

In [ ]:
arr = os.listdir('./outputs/')
print(arr)

In [ ]:
# register the best model

best_model_registered = best_automl_run.register_model(
                                            model_name = "best_automl_covid19_model",
                                            tags={'Area': "Pandemic", 'Type': "Classification",'Method of execution':'Auto ML'},
                                            properties={'Accuracy': best_run_metrics['accuracy']})
#print(best_model.name, best_model.id, sep='\t')
print(best_model_registered)

## Model Deployment

As the best model coming from AutoML run has better accuracy than the one coming from the HyperDrive run, I deploy it in the cell below, register it, create an inference config and deploy the model as a web service.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
with open('score.py') as f:
    print(f.read())

In [ ]:
model = Model(ws, 'best_automl_covid19_model')

environment = best_automl_run.get_environment()
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')
best_automl_run.download_file(constants.CONDA_ENV_FILE_PATH, 'environment.yml')


inference_config = InferenceConfig(entry_script='score.py', environment=environment)

deployment_config = AciWebservice.deploy_configuration(
                                                       cpu_cores=1,
                                                       memory_gb=1,
                                                       description='Covid19 new cases prediction',
                                                       auth_enabled=True,
                                                       enable_app_insights= True,
                                                       collect_model_data = True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True)
service.validate_configuration()
service.wait_for_deployment(show_output=True)

print(service.state)
service.print_deploy_configuration()

In [ ]:
print("State : "+service.state)
print("Key " + service.get_keys()[0])
print("Swagger URI : "+service.swagger_uri)
print("Scoring URI : "+service.scoring_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
#Importing the dataset for testing 

Covid19InfectionsSample = df.sample(2) 
y_df = Covid19InfectionsSample["new_cases"]
Covid19InfectionsSample.drop(['new_cases'], inplace=True, axis=1)
x_df = Covid19InfectionsSample

Covid19DataTesting= json.dumps({'data': x_df.to_dict(orient='records')})

print(Covid19DataTesting)

In [ ]:
headers = {'Content-type': 'application/json'}
headers['Authorization'] = f'Bearer {service.get_keys()[0]}'
# Make the request and display the response
response = requests.post(service.scoring_uri, Covid19DataTesting, headers=headers)
print('Prediction :', response.text)

# Print original labels
print('True Values :', y_df.values)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:
#service.delete()
#cpu_cluster.delete()